<a href="https://colab.research.google.com/github/charlaburnett/change_detection_ukraine/blob/main/change_detection_urkaine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>